In [2]:
# Import library yang diperlukan
import os
import numpy as np
from google.colab import drive
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.mixed_precision import set_global_policy
import tensorflow as tf

In [3]:
# Aktifkan mixed precision untuk mempercepat pelatihan jika menggunakan GPU modern
set_global_policy('mixed_float16')

In [4]:
# Periksa apakah TensorFlow menggunakan GPU
print("TensorFlow version:", tf.__version__)
print("Devices available:", tf.config.list_physical_devices())
if tf.test.gpu_device_name():
    print("GPU is available and working!")
else:
    print("Using CPU, training might be slower.")

TensorFlow version: 2.17.1
Devices available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU is available and working!


In [5]:
# Mounting Google Drive in Colab
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Direktori dataset skintone
skintone_directory = '/content/drive/MyDrive/smartfit/dataset/skintone'

In [7]:
# Buat generator untuk memuat data dari direktori
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalisasi pixel ke rentang [0, 1]
    validation_split=0.2,  # 20% data untuk validation
    rotation_range=20,  # Data augmentasi
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [8]:
# Generator untuk training data
train_generator = datagen.flow_from_directory(
    skintone_directory,
    target_size=(128, 128),  # Ukuran gambar lebih kecil untuk CPU/GPU
    batch_size=16,  # Batch size kecil untuk efisiensi memori
    class_mode='sparse',  # Sparse untuk multi-class classification
    subset='training'
)

Found 26650 images belonging to 4 classes.


In [9]:
# Generator untuk validation data
val_generator = datagen.flow_from_directory(
    skintone_directory,
    target_size=(128, 128),
    batch_size=16,
    class_mode='sparse',
    subset='validation'
)

Found 6660 images belonging to 4 classes.


In [10]:
# Tampilkan informasi dataset
print("Training classes:", train_generator.class_indices)
print("Number of training samples:", train_generator.samples)
print("Number of validation samples:", val_generator.samples)

Training classes: {'dark': 0, 'light': 1, 'mid-dark': 2, 'mid-light': 3}
Number of training samples: 26650
Number of validation samples: 6660


In [11]:
# Gunakan MobileNetV2 sebagai base model (transfer learning)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [12]:
# Tambahkan layer Dense untuk klasifikasi
x = base_model.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
output = Dense(len(train_generator.class_indices), activation='softmax')(x)

In [13]:
# Bangun model
model = Model(inputs=base_model.input, outputs=output)

In [14]:
# Freeze layer base model
for layer in base_model.layers:
    layer.trainable = False

In [16]:
from tensorflow.keras.optimizers import Adam

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Tampilkan summary model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 128, 128, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 64, 64, 32)     │            864 │ cast[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 64, 64, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 64, 64, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 64, 64, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 64, 64, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 64, 64, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 64, 64, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 64, 64, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 64, 64, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 65, 65, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 32, 32, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)    

 Total params: 3,569,028 (13.61 MB)

 Trainable params: 1,311,044 (5.00 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [17]:
# Callback untuk Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,  # Berhenti jika tidak ada peningkatan selama 2 epoch
    restore_best_weights=True
)

In [19]:
# Latih model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=200,  # Gunakan subset batch untuk mempercepat pelatihan
    epochs=3,  # Kurangi jumlah epoch
    validation_data=val_generator,
    validation_steps=50,  # Batasi batch untuk validasi
    callbacks=[early_stopping]
)

Epoch 1/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 3304s 16s/step - accuracy: 0.4181 - loss: 1.2368 - val_accuracy: 0.5638 - val_loss: 0.9582
Epoch 2/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 2810s 14s/step - accuracy: 0.5017 - loss: 1.0681 - val_accuracy: 0.6375 - val_loss: 0.8707
Epoch 3/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 2420s 12s/step - accuracy: 0.5238 - loss: 1.0295 - val_accuracy: 0.6413 - val_loss: 0.8329


In [ ]:
# Evaluasi model pada data validation
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

 11/417 ━━━━━━━━━━━━━━━━━━━━ 56:24 8s/step - accuracy: 0.5929 - loss: 0.9283

In [ ]:
# Simpan model ke file
model.save('/content/drive/MyDrive/smartfit/models/skintone_model_test.h5')
print("Model saved successfully!")